# Chapter 3: Parallelization

Key Takeaways:
- Parallelization allows agents to perform multiple independent tasks simultaneously.
- This reduces overall latency compared to sequential execution.
- It is useful for aggregating different perspectives or processing distinct aspects of a problem.
- In LangChain, `RunnableParallel` (also known as the `Map` step) facilitates this pattern.

### Heuristic: *Many hands make light work.*

## Setup and Initialization

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

In [ ]:
# --- Configuration ---
# Ensure your API key environment variable is set (e.g., GOOGLE_API_KEY)
from dotenv import load_dotenv
load_dotenv()

try:
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
    print(f"Language model initialized: {llm.model}")
except Exception as e:
    print(f"Error initializing language model: {e}")
    llm = None

## Define Independent Chains

We define three distinct chains that can run independently:
1. **Summary Chain**: Summarizes the topic.
2. **Questions Chain**: Generates related questions.
3. **Terms Chain**: Identifies key terms.

In [ ]:
# 1. Summary Chain
summary_template = ChatPromptTemplate.from_messages([
    ("system", "Summarize the following topic concisely:"),
    ("user", "{topic}")
])
if llm:
    summarize_chain = summary_template | llm | StrOutputParser()
else:
    summarize_chain = None

# 2. Questions Chain
questions_template = ChatPromptTemplate.from_messages([
    ("system", "Generate three interesting questions about the following topic:"),
    ("user", "{topic}")
])
if llm:
    questions_chain = questions_template | llm | StrOutputParser()
else:
    questions_chain = None

# 3. Terms Chain
terms_template = ChatPromptTemplate.from_messages([
    ("system", "Identify 5-10 key terms from the following topic, separated by commas:"),
    ("user", "{topic}")
])
if llm:
    terms_chain = terms_template | llm | StrOutputParser()
else:
    terms_chain = None

## Build the Parallel Chain (Map Step)

The `RunnableParallel` component executes the defined chains simultaneously. It takes the input (topic) and passes it to each chain.

In [ ]:
if summarize_chain and questions_chain and terms_chain:
    map_chain = RunnableParallel({
        "summary": summarize_chain,
        "questions": questions_chain,
        "key_terms": terms_chain,
        "topic": RunnablePassthrough()  # Pass the original topic through
    })
else:
    map_chain = None

## Define Synthesis Step (Reduce Step)

Combine the outputs from the parallel execution into a final coherent response.

In [ ]:
synthesis_prompt = ChatPromptTemplate.from_messages([
    ("system", """Based on the following information:

Summary: {summary}

Related Questions: {questions}

Key Terms: {key_terms}

Synthesize a comprehensive answer."""),
    ("user", "Original topic: {topic}")
])

if map_chain and llm:
    full_parallel_chain = map_chain | synthesis_prompt | llm | StrOutputParser()
else:
    full_parallel_chain = None

## Example Usage

Run the full chain with a sample topic.

In [ ]:
topic = "The history of space exploration"

print(f"--- Processing topic: '{topic}' ---")
if full_parallel_chain:
    try:
        result = full_parallel_chain.invoke(topic)
        print("\n--- Final Synthesized Response ---")
        print(result)
    except Exception as e:
        print(f"Error during execution: {e}")
else:
    print("Full chain not initialized.")

## Conclusion

This notebook demonstrated **Parallelization** using `RunnableParallel` in LangChain.

Key benefits:
- **Efficiency**: Running independent tasks concurrently reduces total execution time.
- **Modularity**: Individual chains can be developed and tested separately.
- **Rich Synthesis**: Aggregating diverse outputs leads to more comprehensive final results.